In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
#from scipy import interp, arange, exp
from ccfj import GetStationPairs
import ccfj
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
import shutil
import glob

In [ ]:
import sys
sys.path.append('../tools_F-J')
from toollib_voro import PointinPolygon

In [ ]:
with open('a-project_voro.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/output_repar_03-01/'               # Harmon server
name_project

In [ ]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

In [ ]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [ ]:
dir_stack = dir_project+info_basic['rdir_stack']
dir_CC = dir_CC_workspace+info_basic['rdira_CC']
dir_ds = dir_project + info_basic['rdir_ds']
dir_inv_dispernet = dir_project + info_basic['dir_inv_dispernet']
stalistname_all = info_basic['stalistname_all']
outdir = dir_inv_dispernet + 'data'

In [ ]:
# Generate Target Points matrix X,Y
nx = 40
ny = 40
x = np.linspace(-116.596436,-116.586189,nx)
y = np.linspace(33.535305,33.54248,ny)
X,Y = np.meshgrid(x,y)
X = X.reshape(-1)
Y = Y.reshape(-1)
n = len(X)
xv = [-116.596436, -116.59211, -116.586189, -116.590622]
yv = [33.53941, 33.535305, 33.538368, 33.54248]

In [ ]:
# Set the minima sample times
min_number = 10

In [ ]:
## reading the Voronoi Partitions
dsfiles = glob.glob(dir_ds+'/*h5')
files = []
for i in range(n):
	files.append([])
for j in range(len(dsfiles)):
	h5file = h5py.File(dsfiles[j],'r')
	xvyv = h5file['xvyv'][:]
	h5file.close()
	_in,_on = PointinPolygon.inpolygon(X, Y, xvyv[0,:], xvyv[1,:])
	indx = np.array([i for i,x in enumerate(_in) if x])
	for i in indx:
		files[i].append(j)

In [ ]:
## reading the dispersion curves
curves = []
for fr in dsfiles:
	fr1 = os.path.join(dir_inv_dispernet + 'curves/',fr.split('/')[-1].split('.')[0]+'.txt')
	curve = np.loadtxt(fr1)
	curves.append(curve)

In [ ]:
# estimate the dispersion curves of each target points
for i in range(n):
	print(i)
	_in,_on = PointinPolygon.inpolygon(X[i],Y[i],xv,yv)
	if not _in:
		continue
	fid = open(os.path.join(outdir,'curve'+str(i)+'.txt'),'w')
	for modei in range(1):
		cs = []
		for j in files[i]:
			curve = curves[j]
			if len(curve[curve[:,2]==modei,2])>0:
				cs.append(curve[curve[:,2]==modei,:])
	
		minf = 10
		maxf = 0
		for csi in cs:
			if minf>np.min(csi[:,0]):
				minf = np.min(csi[:,0])

			if maxf <np.max(csi[:,0]):
				maxf = np.max(csi[:,0])
		nf = int((maxf-minf)/0.1) + 1
		for k in range(nf):
			c = []
			f = np.round(minf + k*0.1,2)
			for csi in cs:
				indx = np.where(np.abs(csi[:,0]-f)<0.01)[0]
				if len(indx) > 0:
					c.append(csi[indx,1])
			c = np.asarray(c)/1e3
			if len(c) > min_number:
				mean = np.round(np.mean(c),3)
				std = np.round(np.std(c),3)
				fid.write(str(f)+' '+str(mean)+' '+str(modei)+' '+str(std)+'\n')
	fid.close()